In [1]:
import nltk
import pycrfsuite
from nltk.corpus import conll2002
from nltk.tag import CRFTagger
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('punkt') # Tokenitzador
nltk.download('averaged_perceptron_tagger') # Etiquetador POS
nltk.download('maxent_ne_chunker') # Etiquetador Entitats Anomenades
nltk.download('words')
nltk.download('treebank')
nltk.download('conll2002')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-d

True

In [3]:
train_esp = conll2002.iob_sents('esp.train') # Train
testa_esp = conll2002.iob_sents('esp.testa') # Dev
testb_esp = conll2002.iob_sents('esp.testb') # Test
train_ned = conll2002.iob_sents('ned.train') # Train
testa_ned = conll2002.iob_sents('ned.testa') # Dev
testb_ned = conll2002.iob_sents('ned.testb') # Test

In [4]:
model_tagger_POS = CRFTagger()

In [5]:
import unicodedata
import re

class FeatureExtractor:
    def __init__(self, use_basic_features=False, use_prefix_suffix_features=False, use_context_features=False, pattern = r'\d+'):
        self.use_basic_features = use_basic_features
        self.use_prefix_suffix_features = use_prefix_suffix_features
        self.use_context_features = use_context_features
        self._pattern = pattern
        
    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags
            - longitud

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
            
        token = tokens[idx]
        
        feature_list = []
        
        if self.use_basic_features:
            # Capitalization
            if token[0].isupper():
                feature_list.append("CAPITALIZATION")

            # Number
            if re.search(self._pattern, token) is not None:
                feature_list.append("HAS_NUM")

            # Punctuation
            punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
            if all(unicodedata.category(x) in punc_cat for x in token):
                feature_list.append("PUNCTUATION")
                    
        if self.use_prefix_suffix_features:
            # preffix up to length 3
            if len(token) > 1:
                feature_list.append("PRE_" + token[:1])
            if len(token) > 2:
                feature_list.append("PRE_" + token[:2])
            if len(token) > 3:
                feature_list.append("PRE_" + token[:3])

            # Suffix up to length 3
            if len(token) > 1:
                feature_list.append("SUF_" + token[-1:])
            if len(token) > 2:
                feature_list.append("SUF_" + token[-2:])
            if len(token) > 3:
                feature_list.append("SUF_" + token[-3:])
    
        
        if self.use_context_features:
            # POS_tags
            POS = model_tagger_POS.tag(tokens)
                
            # Paraules prèvies amb POS
            if idx > 0:
                feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
            if idx > 1:
                feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
                
            # Paraules posteriors amb POS
            if idx < (len(tokens)-1):
                feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
            if idx < (len(tokens)-2):
                feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

            feature_list.append("WORD_" + token)
            
        
        return feature_list

In [16]:
def obtener_token(fitxer):
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append(elem1)
        res.append(frases)
    return res

## Predicció amb BIO

In [6]:
"""def obtener_token_POS(fitxer):
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append((elem1, elem2))
        res.append(frases)
    return res"""

def obtener_token_BIO(fitxer):
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append((elem1, elem3))
        res.append(frases)
    return res    

In [7]:
# Avaluació amb sets veure si son iguals també es pot mirar per intersecció i si coincideixen
def obtener_entidades_con_posiciones_prova(testa_esp_BIO_tag):
    entitats_con_posiciones = set()

    for sentence_index, sentence in enumerate(testa_esp_BIO_tag):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual
        prev_tag = None  # Almacenar la etiqueta del token anterior

        for token_index, token in enumerate(sentence):
            word, tag = token
            #word = word[0]

            if tag.startswith('B-'):
                # Si hay una entidad anterior, la agregamos a la lista de entidades
                if ent:
                    end_pos = token_index - 1  # La posición de fin es el token anterior
                    entitats_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                # Creamos una nueva entidad con la palabra actual
                ent = [word]
                # Obtenemos el tipo de entidad
                name = tag.split('-')[1]
                start_pos = token_index  # La posición de inicio es el token actual
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
            elif tag.startswith('I-'):
                # Solo agregamos la palabra actual si el token anterior tiene etiqueta I- o B-
                if prev_tag:
                    ent.append(word)
                    prev_tag = tag  # Actualizamos la etiqueta del token anterior
            elif tag == 'O' and ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entitats_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []
                prev_tag = None  # Reiniciamos la etiqueta del token anterior

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entitats_con_posiciones.add((tuple(ent), (start_pos, end_pos)))

    return entitats_con_posiciones

In [8]:
def calcular_precision(entidades_referencia, entidades_extraidas):
    # Calcular el número de entidades correctamente extraídas
    entidades_correctas = entidades_referencia.intersection(entidades_extraidas)
    
    # Calcular la precisión
    if len(entidades_extraidas) > 0:
        precision = len(entidades_correctas) / len(entidades_extraidas)
    else:
        precision = 0.0
    
    return precision


def calcular_exhaustividad(entidades_referencia, entidades_extraidas):
    # Calcular el número de entidades correctamente extraídas
    entidades_correctas = entidades_referencia.intersection(entidades_extraidas)
    
    # Calcular la exhaustividad
    if len(entidades_referencia) > 0:
        exhaustividad = len(entidades_correctas) / len(entidades_referencia)
    else:
        exhaustividad = 0.0
    
    return exhaustividad

def calcular_f1_score(precision, exhaustividad):
    # Calcular el F1-score
    if (precision + exhaustividad) > 0:
        f1_score = 2 * (precision * exhaustividad) / (precision + exhaustividad)
    else:
        f1_score = 0.0
    
    return f1_score

def resultats(predicted_BIO, testa_esp_BIO_tag):

    # Obtener los conjuntos de entidades de referencia y extraídas
    entidades_referencia = obtener_entidades_con_posiciones_prova(testa_esp_BIO_tag)  # Conjunto de entidades etiquetadas manualmente como referencia
    entidades_extraidas =  obtener_entidades_con_posiciones_prova(predicted_BIO) # Obtener conjuntos de entidades extraídas

    # Calcular la precisión
    precision = calcular_precision(entidades_referencia, entidades_extraidas)

    # Calcular la exhaustividad
    exhaustividad = calcular_exhaustividad(entidades_referencia, entidades_extraidas)

    # Calcular el F1-score
    f1_score = calcular_f1_score(precision, exhaustividad)

    print("Precisió:", precision)
    print("Exhaustivitat:", exhaustividad)
    print("F1-score:", f1_score)


#### ESP

In [11]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_esp_pre_tag = obtener_token(testa_esp)

train_esp_BIO = obtener_token_BIO(train_esp)

testa_esp_real = obtener_token_BIO(testa_esp)

def model_entrenament(train, extractor, testa):
    
    model_BIO = CRFTagger(feature_func=extractor._get_features)
    model_BIO.train(train, 'model_BIO.crf.tagger')
    
    predicted_BIO = model_BIO.tag_sents(testa)
    
    return resultats(predicted_BIO, testa_esp_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_esp_BIO, param, testa_esp_pre_tag)
    print('-'*50)
    i += 1
    

Model:  1
Precisió: 0.0
Exhaustivitat: 0.0
F1-score: 0.0
--------------------------------------------------
Model:  2
Precisió: 0.39166666666666666
Exhaustivitat: 0.3643410852713178
F1-score: 0.37751004016064255
--------------------------------------------------
Model:  3
Precisió: 0.6701492537313433
Exhaustivitat: 0.6215393133997785
F1-score: 0.6449296179258833
--------------------------------------------------
Model:  4
Precisió: 0.7519742614799649
Exhaustivitat: 0.7117940199335548
F1-score: 0.7313326696060305
--------------------------------------------------


In [13]:
feature_extractor = FeatureExtractor(True, True, True)

testa_esp_real = obtener_token_BIO(testb_esp)
testa_esp_pre_tag = obtener_token(testb_esp)

model_BIO = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO.train(train_esp_BIO, 'model_BIO.crf.tagger')
    
predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)

resultats(predicted_BIO, testa_esp_real)


#### NED

In [12]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_ned_pre_tag = obtener_token(testa_ned)

train_ned_BIO = obtener_token_BIO(train_ned)

testa_ned_real = obtener_token_BIO(testa_ned)

def model_entrenament(train, extractor, testa):
    
    model_BIO = CRFTagger(feature_func=extractor._get_features)
    model_BIO.train(train, 'model_BIO.crf.tagger')
    
    predicted_BIO = model_BIO.tag_sents(testa)
    
    return resultats(predicted_BIO, testa_ned_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_ned_BIO, param, testa_ned_pre_tag)
    print('-'*50)
    i += 1

Model:  1
Precisió: 0.6923076923076923
Exhaustivitat: 0.01637852593266606
F1-score: 0.032
--------------------------------------------------
Model:  2
Precisió: 0.26930409914204007
Exhaustivitat: 0.2570518653321201
F1-score: 0.26303538175046554
--------------------------------------------------
Model:  3
Precisió: 0.642230252968508
Exhaustivitat: 0.5659690627843494
F1-score: 0.6016928657799274
--------------------------------------------------
Model:  4
Precisió: 0.7045231071779744
Exhaustivitat: 0.6519563239308462
F1-score: 0.6772211720226843
--------------------------------------------------


In [15]:
feature_extractor = FeatureExtractor(True, True, True)

testa_ned_real = obtener_token_BIO(testb_ned)
testa_ned_pre_tag = obtener_token(testb_ned)

model_BIO = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO.train(train_ned_BIO, 'model_BIO.crf.tagger')
    
predicted_BIO = model_BIO.tag_sents(testa_ned_pre_tag)

resultats(predicted_BIO, testa_ned_real)

Precisió: 0.7329790767186981
Exhaustivitat: 0.6759571209800919
F1-score: 0.7033142128744423


## Predicció amb IO

In [17]:
def convert_to_io(train_data_bio):
    train_data_io = []
    for sentence in train_data_bio:
        io_tags = []
        for word, pos_tag, bio_tag in sentence:
            if bio_tag == 'O':
                io_tags.append('O')
            elif bio_tag.startswith('B-'):
                io_tags.append('I' + bio_tag[1:])
            else:
                io_tags.append(bio_tag)
                
        train_data_io.append(list(zip([word for word, pos_tag, bio_tag in sentence], io_tags)))
    return train_data_io

In [18]:
def obtener_entidades_con_posiciones_io(train_data_io):
    entidades_con_posiciones = set()

    for sentence_index, sentence in enumerate(train_data_io):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual
        prev_tag = None  # Almacenar la etiqueta del token anterior

        for token_index, (word, io_tag) in enumerate(sentence):
            if io_tag != 'O':
                if io_tag.startswith('I-'):
                    ent.append(word)
                else:
                    # Si hay una entidad anterior, la agregamos a la lista de entidades
                    if ent:
                        end_pos = token_index - 1  # La posición de fin es el token anterior
                        entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                    # Creamos una nueva entidad con la palabra actual
                    ent = [word]
                    # Obtenemos el tipo de entidad
                    name = io_tag.split('-')[1]
                    start_pos = token_index  # La posición de inicio es el token actual
                prev_tag = io_tag  # Actualizamos la etiqueta del token anterior
            elif ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []
                prev_tag = None  # Reiniciamos la etiqueta del token anterior

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))

    return entidades_con_posiciones


In [19]:
def calcular_precision(entidades_referencia, entidades_extraidas):
    # Calcular el número de entidades correctamente extraídas
    entidades_correctas = entidades_referencia.intersection(entidades_extraidas)
    
    # Calcular la precisión
    if len(entidades_extraidas) > 0:
        precision = len(entidades_correctas) / len(entidades_extraidas)
    else:
        precision = 0.0
    
    return precision


def calcular_exhaustividad(entidades_referencia, entidades_extraidas):
    # Calcular el número de entidades correctamente extraídas
    entidades_correctas = entidades_referencia.intersection(entidades_extraidas)
    
    # Calcular la exhaustividad
    if len(entidades_referencia) > 0:
        exhaustividad = len(entidades_correctas) / len(entidades_referencia)
    else:
        exhaustividad = 0.0
    
    return exhaustividad

def calcular_f1_score(precision, exhaustividad):
    # Calcular el F1-score
    if (precision + exhaustividad) > 0:
        f1_score = 2 * (precision * exhaustividad) / (precision + exhaustividad)
    else:
        f1_score = 0.0
    
    return f1_score

def resultats(predicted_BIO, testa_esp_IO_tag):

    # Obtener los conjuntos de entidades de referencia y extraídas
    entidades_referencia = obtener_entidades_con_posiciones_io(testa_esp_IO_tag)  # Conjunto de entidades etiquetadas manualmente como referencia
    entidades_extraidas =  obtener_entidades_con_posiciones_io(predicted_BIO) # Obtener conjuntos de entidades extraídas

    # Calcular la precisión
    precision = calcular_precision(entidades_referencia, entidades_extraidas)

    # Calcular la exhaustividad
    exhaustividad = calcular_exhaustividad(entidades_referencia, entidades_extraidas)

    # Calcular el F1-score
    f1_score = calcular_f1_score(precision, exhaustividad)

    print("Precisió:", precision)
    print("Exhaustivitat:", exhaustividad)
    print("F1-score:", f1_score)


#### ESP

In [25]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_esp_pre_tag = obtener_token(testa_esp)

train_esp_IO = convert_to_io(train_esp)

testa_esp_real = convert_to_io(testa_esp)

def model_entrenament(train, extractor, testa):
    
    model_IO = CRFTagger(feature_func=extractor._get_features)
    model_IO.train(train, 'model_IO.crf.tagger')
    
    predicted_IO = model_IO.tag_sents(testa)
    
    return resultats(predicted_IO, testa_esp_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_esp_IO, param, testa_esp_pre_tag)
    print('-'*50)
    i += 1
    

Model:  1
Precisió: 0.0
Exhaustivitat: 0.0
F1-score: 0.0
--------------------------------------------------
Model:  2
Precisió: 0.771102433163112
Exhaustivitat: 0.7271954674220963
F1-score: 0.7485056130631288
--------------------------------------------------
Model:  3
Precisió: 0.8323556653288052
Exhaustivitat: 0.7637393767705383
F1-score: 0.7965726104299009
--------------------------------------------------
Model:  4
Precisió: 0.9115566037735849
Exhaustivitat: 0.8759206798866855
F1-score: 0.8933834151979198
--------------------------------------------------


In [27]:
feature_extractor = FeatureExtractor(True, True, True)

testa_esp_real = convert_to_io(testb_esp)
testa_esp_pre_tag = obtener_token(testb_esp)

model_IO = CRFTagger(feature_func=feature_extractor._get_features)
model_IO.train(train_esp_IO, 'model_IO.crf.tagger')
    
predicted_IO = model_IO.tag_sents(testa_esp_pre_tag)

resultats(predicted_IO, testa_esp_real)


Precisió: 0.9248455730954015
Exhaustivitat: 0.896242101762554
F1-score: 0.9103192028373586


#### NED

In [28]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_ned_pre_tag = obtener_token(testa_ned)

train_ned_IO = convert_to_io(train_ned)

testa_ned_real = convert_to_io(testa_ned)

def model_entrenament(train, extractor, testa):
    
    model_IO = CRFTagger(feature_func=extractor._get_features)
    model_IO.train(train, 'model_IO.crf.tagger')
    
    predicted_IO = model_IO.tag_sents(testa)
    
    return resultats(predicted_IO, testa_ned_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_ned_IO, param, testa_ned_pre_tag)
    print('-'*50)
    i += 1

Model:  1
Precisió: 0.0
Exhaustivitat: 0.0
F1-score: 0.0
--------------------------------------------------
Model:  2
Precisió: 0.5677352637021716
Exhaustivitat: 0.25884016973125884
F1-score: 0.3555699481865285
--------------------------------------------------
Model:  3
Precisió: 0.8836594394500265
Exhaustivitat: 0.7878359264497878
F1-score: 0.8330009970089731
--------------------------------------------------
Model:  4
Precisió: 0.9229198570699336
Exhaustivitat: 0.8524280999528524
F1-score: 0.8862745098039216
--------------------------------------------------


In [29]:
feature_extractor = FeatureExtractor(True, True, True)

testa_ned_real = convert_to_io(testb_ned)
testa_ned_pre_tag = obtener_token(testb_ned)

model_IO = CRFTagger(feature_func=feature_extractor._get_features)
model_IO.train(train_ned_IO, 'model_IO.crf.tagger')
    
predicted_IO = model_IO.tag_sents(testa_ned_pre_tag)

resultats(predicted_IO, testa_ned_real)

Precisió: 0.9179993194964273
Exhaustivitat: 0.8575969485060394
F1-score: 0.8867707477403451


## Predicció amb BIOW

In [30]:
def convert_to_biow(train_data_bio):
    train_data_biow = []
    for sentence in train_data_bio:
        biow_tags = []
        for word, pos_tag, bio_tag in sentence:
            if bio_tag == 'O':
                biow_tags.append('O')
            else:
                biow_tags.append(bio_tag + 'W')  # Añadir 'W' a todas las etiquetas
            
        train_data_biow.append(list(zip([word for word, pos_tag, bio_tag in sentence], biow_tags)))
    return train_data_biow

In [34]:
def obtener_entidades_con_posiciones_biow(train_data_biow):
    entidades_con_posiciones = set()

    for sentence_index, sentence in enumerate(train_data_biow):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual
        prev_tag = None  # Almacenar la etiqueta del token anterior

        for token_index, (word, biow_tag) in enumerate(sentence):
            if biow_tag != 'O':
                if biow_tag.endswith('W'):
                    # Eliminar 'W' del final de la etiqueta
                    io_tag = biow_tag[:-1]
                else:
                    io_tag = biow_tag
                if io_tag.startswith('I-'):
                    ent.append(word)
                else:
                    # Si hay una entidad anterior, la agregamos a la lista de entidades
                    if ent:
                        end_pos = token_index - 1  # La posición de fin es el token anterior
                        entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                    # Creamos una nueva entidad con la palabra actual
                    ent = [word]
                    # Obtenemos el tipo de entidad
                    name = io_tag.split('-')[1]
                    start_pos = token_index  # La posición de inicio es el token actual
                prev_tag = io_tag  # Actualizamos la etiqueta del token anterior
            elif ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []
                prev_tag = None  # Reiniciamos la etiqueta del token anterior

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))

    return entidades_con_posiciones


In [35]:
def calcular_precision(entidades_referencia, entidades_extraidas):
    # Calcular el número de entidades correctamente extraídas
    entidades_correctas = entidades_referencia.intersection(entidades_extraidas)
    
    # Calcular la precisión
    if len(entidades_extraidas) > 0:
        precision = len(entidades_correctas) / len(entidades_extraidas)
    else:
        precision = 0.0
    
    return precision


def calcular_exhaustividad(entidades_referencia, entidades_extraidas):
    # Calcular el número de entidades correctamente extraídas
    entidades_correctas = entidades_referencia.intersection(entidades_extraidas)
    
    # Calcular la exhaustividad
    if len(entidades_referencia) > 0:
        exhaustividad = len(entidades_correctas) / len(entidades_referencia)
    else:
        exhaustividad = 0.0
    
    return exhaustividad

def calcular_f1_score(precision, exhaustividad):
    # Calcular el F1-score
    if (precision + exhaustividad) > 0:
        f1_score = 2 * (precision * exhaustividad) / (precision + exhaustividad)
    else:
        f1_score = 0.0
    
    return f1_score

def resultats(predicted_BIO, testa_esp_IO_tag):

    # Obtener los conjuntos de entidades de referencia y extraídas
    entidades_referencia = obtener_entidades_con_posiciones_biow(testa_esp_IO_tag)  # Conjunto de entidades etiquetadas manualmente como referencia
    entidades_extraidas =  obtener_entidades_con_posiciones_biow(predicted_BIO) # Obtener conjuntos de entidades extraídas

    # Calcular la precisión
    precision = calcular_precision(entidades_referencia, entidades_extraidas)

    # Calcular la exhaustividad
    exhaustividad = calcular_exhaustividad(entidades_referencia, entidades_extraidas)

    # Calcular el F1-score
    f1_score = calcular_f1_score(precision, exhaustividad)

    print("Precisió:", precision)
    print("Exhaustivitat:", exhaustividad)
    print("F1-score:", f1_score)


#### ESP

In [36]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_esp_pre_tag = obtener_token(testa_esp)

train_esp_BIOW = convert_to_biow(train_esp)

testa_esp_real = convert_to_biow(testa_esp)

def model_entrenament(train, extractor, testa):
    
    model_BIOW = CRFTagger(feature_func=extractor._get_features)
    model_BIOW.train(train, 'model_BIOW.crf.tagger')
    
    predicted_BIOW = model_BIOW.tag_sents(testa)
    
    return resultats(predicted_BIOW, testa_esp_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_esp_BIOW, param, testa_esp_pre_tag)
    print('-'*50)
    i += 1
    

Model:  1
Precisió: 0.0
Exhaustivitat: 0.0
F1-score: 0.0
--------------------------------------------------
Model:  2
Precisió: 0.39166666666666666
Exhaustivitat: 0.36424024356490453
F1-score: 0.37745590133371576
--------------------------------------------------
Model:  3
Precisió: 0.6693500298151461
Exhaustivitat: 0.6213672848048712
F1-score: 0.6444667719247883
--------------------------------------------------
Model:  4
Precisió: 0.7517543859649123
Exhaustivitat: 0.7115970107943537
F1-score: 0.7311246978529788
--------------------------------------------------


In [37]:
feature_extractor = FeatureExtractor(True, True, True)

testa_esp_real = convert_to_biow(testb_esp)
testa_esp_pre_tag = obtener_token(testb_esp)

model_BIOW = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOW.train(train_esp_BIOW, 'model_BIOW.crf.tagger')
    
predicted_BIOW = model_BIOW.tag_sents(testa_esp_pre_tag)

resultats(predicted_BIOW, testa_esp_real)


Precisió: 0.7885268160217244
Exhaustivitat: 0.7656558998022412
F1-score: 0.7769230769230768


#### NED

In [38]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_ned_pre_tag = obtener_token(testa_ned)

train_ned_BIOW = convert_to_biow(train_ned)

testa_ned_real = convert_to_biow(testa_ned)

def model_entrenament(train, extractor, testa):
    
    model_BIOW = CRFTagger(feature_func=extractor._get_features)
    model_BIOW.train(train, 'model_BIOW.crf.tagger')
    
    predicted_BIOW = model_BIOW.tag_sents(testa)
    
    return resultats(predicted_BIOW, testa_ned_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_ned_BIOW, param, testa_ned_pre_tag)
    print('-'*50)
    i += 1

Model:  1
Precisió: 0.5769230769230769
Exhaustivitat: 0.013692377909630305
F1-score: 0.026749888542131076
--------------------------------------------------
Model:  2
Precisió: 0.2477326968973747
Exhaustivitat: 0.2368781378366043
F1-score: 0.24218385440970602
--------------------------------------------------
Model:  3
Precisió: 0.6316606311433006
Exhaustivitat: 0.5572797809219534
F1-score: 0.5921435499515034
--------------------------------------------------
Model:  4
Precisió: 0.6983735830458354
Exhaustivitat: 0.6467366499315381
F1-score: 0.6715639810426541
--------------------------------------------------


In [40]:
feature_extractor = FeatureExtractor(True, True, True)

testa_ned_real = convert_to_biow(testb_ned)
testa_ned_pre_tag = obtener_token(testb_ned)

model_BIOW = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOW.train(train_ned_BIOW, 'model_BIOW.crf.tagger')
    
predicted_BIOW = model_BIOW.tag_sents(testa_ned_pre_tag)

resultats(predicted_BIOW, testa_ned_real)

Precisió: 0.7248165443629087
Exhaustivitat: 0.6684097200861273
F1-score: 0.6954712754040647


## Predicció amb BIOES

In [41]:
def convert_to_bioes(train_data_bio):
    train_data_bioes = []
    for sentence in train_data_bio:
        bioes_tags = []
        for i, (word, pos_tag, bio_tag) in enumerate(sentence):
            if bio_tag == 'O':
                bioes_tags.append('O')
            elif bio_tag.startswith('B-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('S' + bio_tag[1:])  # Single
                else:
                    bioes_tags.append('B' + bio_tag[1:])  # Begin
            elif bio_tag.startswith('I-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('E' + bio_tag[1:])  # End
                else:
                    bioes_tags.append('I' + bio_tag[1:])  # Inside
            else:
                raise ValueError("Etiqueta BIO incorrecta: {}".format(bio_tag))
                
        train_data_bioes.append(list(zip([word for word, pos_tag, bio_tag in sentence], bioes_tags)))
    return train_data_bioes

In [42]:
def obtener_entidades_con_posiciones_bioes(train_data_bioes):
    entidades_con_posiciones = set()

    for sentence_index, sentence in enumerate(train_data_bioes):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual

        for token_index, (word, bioes_tag) in enumerate(sentence):
            if bioes_tag != 'O':
                if bioes_tag.startswith('B-') or bioes_tag.startswith('S-'):
                    # Si hay una entidad anterior, la agregamos a la lista de entidades
                    if ent:
                        end_pos = token_index - 1  # La posición de fin es el token anterior
                        entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                    # Creamos una nueva entidad con la palabra actual
                    ent = [word]
                    # Obtenemos el tipo de entidad
                    name = bioes_tag.split('-')[1]
                    start_pos = token_index  # La posición de inicio es el token actual
                elif bioes_tag.startswith('I-') or bioes_tag.startswith('E-'):
                    ent.append(word)
            elif ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entidades_con_posiciones.add((tuple(ent), (start_pos, end_pos), name))

    return entidades_con_posiciones


#### ESP

In [43]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_esp_pre_tag = obtener_token(testa_esp)

train_esp_BIOES = convert_to_bioes(train_esp)

testa_esp_real = convert_to_bioes(testa_esp)

def model_entrenament(train, extractor, testa):
    
    model_BIOES = CRFTagger(feature_func=extractor._get_features)
    model_BIOES.train(train, 'model_BIOW.crf.tagger')
    
    predicted_BIOES = model_BIOES.tag_sents(testa)
    
    return resultats(predicted_BIOES, testa_esp_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_esp_BIOES, param, testa_esp_pre_tag)
    print('-'*50)
    i += 1
    

Model:  1
Precisió: 0.0
Exhaustivitat: 0.0
F1-score: 0.0
--------------------------------------------------
Model:  2
Precisió: 0.4167164179104478
Exhaustivitat: 0.4073929961089494
F1-score: 0.4120019675356616
--------------------------------------------------
Model:  3
Precisió: 0.7058585039529698
Exhaustivitat: 0.677431906614786
F1-score: 0.6913531222078825
--------------------------------------------------
Model:  4
Precisió: 0.7671623548584233
Exhaustivitat: 0.7326848249027237
F1-score: 0.7495273161508608
--------------------------------------------------


In [44]:
feature_extractor = FeatureExtractor(True, True, True)

testa_esp_real = convert_to_bioes(testb_esp)
testa_esp_pre_tag = obtener_token(testb_esp)

model_BIOES = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES.train(train_esp_BIOES, 'model_BIOES.crf.tagger')
    
predicted_BIOES = model_BIOES.tag_sents(testa_esp_pre_tag)

resultats(predicted_BIOES, testa_esp_real)


Precisió: 0.7937620889748549
Exhaustivitat: 0.7768575485092286
F1-score: 0.7852188471657499


#### NED

In [45]:
from itertools import product

param_combinations = [
    FeatureExtractor(),
    FeatureExtractor(True),
    FeatureExtractor(True, True),
    FeatureExtractor(True, True, True)
]

testa_ned_pre_tag = obtener_token(testa_ned)

train_ned_BIOES = convert_to_bioes(train_ned)

testa_ned_real = convert_to_bioes(testa_ned)

def model_entrenament(train, extractor, testa):
    
    model_BIOES = CRFTagger(feature_func=extractor._get_features)
    model_BIOES.train(train, 'model_BIOES.crf.tagger')
    
    predicted_BIOES = model_BIOES.tag_sents(testa)
    
    return resultats(predicted_BIOES, testa_ned_real)
i = 1
for param in param_combinations:
    print('Model: ', i)
    model_entrenament(train_ned_BIOES, param, testa_ned_pre_tag)
    print('-'*50)
    i += 1

Model:  1
Precisió: 0.6122448979591837
Exhaustivitat: 0.0211864406779661
F1-score: 0.040955631399317405
--------------------------------------------------
Model:  2
Precisió: 0.35880022637238257
Exhaustivitat: 0.22387005649717515
F1-score: 0.2757121113285497
--------------------------------------------------
Model:  3
Precisió: 0.6693672839506173
Exhaustivitat: 0.6126412429378532
F1-score: 0.6397492625368733
--------------------------------------------------
Model:  4
Precisió: 0.7253925698965914
Exhaustivitat: 0.6687853107344632
F1-score: 0.6959397391144589
--------------------------------------------------


In [46]:
feature_extractor = FeatureExtractor(True, True, True)

testa_ned_real = convert_to_bioes(testb_ned)
testa_ned_pre_tag = obtener_token(testb_ned)

model_BIOES = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES.train(train_ned_BIOES, 'model_BIOES.crf.tagger')
    
predicted_BIOES = model_BIOES.tag_sents(testa_ned_pre_tag)

resultats(predicted_BIOES, testa_ned_real)

Precisió: 0.7569790628115654
Exhaustivitat: 0.703660797034291
F1-score: 0.7293467819404418
